In [4]:

"""
Bloom & Petals Flower Shop - AI Business Assistant
"""

import google.generativeai as genai
import gradio as gr
import json
from datetime import datetime

!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/.env')
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)



BUSINESS_SUMMARY = """Bloom & Petals Flower Shop - Business Summary

Bloom & Petals is a premier flower shop dedicated to bringing beauty, joy, and elegance to every occasion.
Located downtown, we create stunning floral arrangements for weddings, corporate events, birthdays, anniversaries,
and everyday celebrations.

Services Offered:
- Custom floral arrangements for all occasions
- Wedding and event floral design
- Corporate floral services and subscriptions
- Same-day delivery within the city
- Flower care workshops
- Gift baskets and seasonal bouquets

Operating Hours:
Mon–Sat: 9 AM – 7 PM
Sun: 10 AM – 5 PM

Contact:
Phone: (555) 123-4567
Email: hello@bloomandpetals.com
Address: 123 Garden Street, Downtown District
"""

print("✅ Business information loaded successfully!")


customer_leads, customer_feedback = [], []

def save_data():
    """Save leads & feedback to disk."""
    with open("customer_leads.json", "w", encoding="utf-8") as f:
        json.dump(customer_leads, f, indent=2)
    with open("customer_feedback.json", "w", encoding="utf-8") as f:
        json.dump(customer_feedback, f, indent=2)

# ============================================================
# 5. Tool Functions
# ============================================================
def record_customer_interest(email: str, name: str, message: str) -> str:
    """Record customer contact info & inquiry."""
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    lead = {"timestamp": ts, "name": name, "email": email, "message": message}
    customer_leads.append(lead)
    save_data()
    print(f"\n📝 NEW LEAD: {lead}")
    return f"Thank you {name}! Your inquiry has been recorded. We'll contact you at {email} soon."

def record_feedback(question: str) -> str:
    """Record customer feedback or unanswered question."""
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    fb = {"timestamp": ts, "question": question}
    customer_feedback.append(fb)
    save_data()
    print(f"\n💬 FEEDBACK: {fb}")
    return "Your feedback has been recorded. Our team will review it soon."

print("✅ Tool functions ready!")

# ============================================================
# 6. Function Declarations for Gemini
# ============================================================
tools = [{
    "function_declarations": [
        {
            "name": "record_customer_interest",
            "description": "Record a customer's inquiry and contact details.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "Customer's email"},
                    "name": {"type": "string", "description": "Customer's name"},
                    "message": {"type": "string", "description": "Inquiry details"},
                },
                "required": ["email", "name", "message"],
            },
        },
        {
            "name": "record_feedback",
            "description": "Record customer feedback when a question cannot be answered.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "Customer's question"},
                },
                "required": ["question"],
            },
        },
    ]
}]

# ============================================================
# 7. System Prompt (forces proper tool use)
# ============================================================
SYSTEM_PROMPT = f"""
You are a friendly AI assistant for Bloom & Petals Flower Shop.

BUSINESS INFORMATION:
{BUSINESS_SUMMARY}

RULES:
1. When a customer provides both their name and email, you MUST call 'record_customer_interest' with:
   - email: customer's email
   - name: customer's name
   - message: their request (e.g., "10 red tulips for anniversary on May 14")
2. If you cannot answer a question, call 'record_feedback' with the question text.
3. Do NOT only reply with text when a function should be called.
4. After executing a function, politely confirm it to the user.
5. Always be warm, enthusiastic, and professional.
"""

# ============================================================
# 8. Model Setup
# ============================================================
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-001",
    tools=tools,
    system_instruction=SYSTEM_PROMPT,
)
print("✅ Gemini model initialized!")

# Persistent chat
chat_session = model.start_chat(history=[])

# ============================================================
# 9. Function Map
# ============================================================
available_functions = {
    "record_customer_interest": record_customer_interest,
    "record_feedback": record_feedback,
}

# ============================================================
# 10. Chat Handler (final fixed)
# ============================================================
def chat_with_agent(message: str, history: list) -> str:
    """Handle chat, detect function calls in any part."""
    try:
        response = chat_session.send_message(message)

        # Debug
        print("\n=== DEBUG RESPONSE ===")
        print(response)
        print("======================\n")

        if not response.candidates:
            return "I'm sorry, I couldn't generate a response."

        cand = response.candidates[0]
        if not cand.content or not cand.content.parts:
            return "I couldn’t understand that. Please try again."

        # 🔍 Loop through all parts to catch function calls
        for part in cand.content.parts:
            if hasattr(part, "function_call") and part.function_call:
                fn_name = part.function_call.name
                fn_args = dict(part.function_call.args)
                print(f"\n🤖 Function call: {fn_name} | Args: {fn_args}")

                if fn_name in available_functions:
                    fn_result = available_functions[fn_name](**fn_args)
                    save_data()
                    chat_session.send_message(f"Function '{fn_name}' executed successfully.")
                    return f"✅ {fn_result}"
                else:
                    return f"⚠️ Function {fn_name} not found."

        # 🗨️ No function call → combine text parts
        text_parts = [p.text for p in cand.content.parts if hasattr(p, "text")]
        return " ".join(text_parts) if text_parts else "How else can I help you?"

    except Exception as e:
        print(f"Error: {e}")
        return "Sorry, something went wrong while processing your request."

print("✅ Chat handler ready!")

# ============================================================
# 11. Gradio Interface
# ============================================================
with gr.Blocks(title="Bloom & Petals AI Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🌸 Bloom & Petals Flower Shop
        ### AI-Powered Customer Assistant

        Welcome! I can help you with:
        - Custom floral orders 🌷
        - Same-day delivery 🚚
        - Event flower planning 💐
        - Shop information 🏵️
        """
    )

    chatbot = gr.ChatInterface(
        fn=chat_with_agent,
        examples=[
            "I'd like to order 10 red tulips for my anniversary on May 14.",
            "Do you make wedding bouquets?",
            "Can you make chocolate-covered strawberries?",
            "Do you offer international delivery?",
        ],
    )

    with gr.Accordion("📊 View Collected Data", open=False):
        gr.Markdown("### Customer Leads")
        leads_display = gr.JSON(label="Leads", value=customer_leads)
        gr.Markdown("### Customer Feedback")
        feedback_display = gr.JSON(label="Feedback", value=customer_feedback)

        refresh_btn = gr.Button("🔄 Refresh Data")
        reset_btn = gr.Button("🧹 Reset Conversation")

        def refresh_data():
            return customer_leads, customer_feedback

        def reset_chat():
            global chat_session
            chat_session = model.start_chat(history=[])
            return "🧾 Conversation reset. How can I assist you today?"

        refresh_btn.click(fn=refresh_data, outputs=[leads_display, feedback_display])
        reset_btn.click(fn=reset_chat, outputs=chatbot)

print("✅ Interface ready!")


demo.launch(share=True, debug=True)


✅ Business information loaded successfully!
✅ Tool functions ready!
✅ Gemini model initialized!
✅ Chat handler ready!


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


✅ Interface ready!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b36aa7677e15040945.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



=== DEBUG RESPONSE ===
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "record_feedback",
                  "args": {
                    "question": "Do you offer international delivery?"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.027634713053703307
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 391,
        "candidates_token_count": 10,
        "total_token_count": 401
      },
      "model_version": "gemini-2.0-flash-001"
    }),
)


🤖 Function call: record_feedback | Args: {'question': 'Do you offer international delivery?'}

💬 FEEDBACK: {'timestamp': '2025-10-19 08:50:03', 'question': 'Do you offer internati

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 